# YOLO Object Detection

## Installation

In [1]:
!git clone https://github.com/AlexeyAB/darknet.git
!chmod +x ./darknet
!ls darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14730, done.
remote: Total 14730 (delta 0), reused 0 (delta 0), pack-reused 14730
Receiving objects: 100% (14730/14730), 13.27 MiB | 23.85 MiB/s, done.
Resolving deltas: 100% (10020/10020), done.
3rdparty	DarknetConfig.cmake.in	include		       results
build		darknet_images.py	json_mjpeg_streams.sh  scripts
build.ps1	darknet.py		LICENSE		       src
build.sh	darknet_video.py	Makefile	       vcpkg.json
cfg		data			net_cam_v3.sh	       video_yolov3.sh
cmake		image_yolov3.sh		net_cam_v4.sh	       video_yolov4.sh
CMakeLists.txt	image_yolov4.sh		README.md


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [4]:
with open('darknet/Makefile', 'r') as file_:
  file_content = file_.read()
file_content = file_content.replace('GPU=0', 'GPU=1').replace('CUDNN=0', 'CUDNN=1').replace('OPENCV=0', 'OPENCV=1')

f = open('darknet/Makefile', 'w')
f.write(file_content)
f.close()

In [5]:
%cd darknet
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

## Training

In [6]:
# Install Kaggle API
!pip install -q kaggle
!pip install -q kaggle-cli

     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 4.2MB 29.5MB/s 
     |████████████████████████████████| 112kB 54.0MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 143kB 61.7MB/s 
  ERROR: Failed building wheel for lxml
    Running setup.py install for lxml ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-2j5dnx37/lxml/setup.py'"'"'; __file__='"'"'/tmp/pip-install-2j5dnx37/lxml/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-8u05tfiq/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [7]:
# only for google colab
import os
os.environ['KAGGLE_USERNAME'] = "<username>" 
os.environ['KAGGLE_KEY'] = "<key>"

In [8]:
!kaggle datasets download -d tannergi/microcontroller-detection --unzip

 84% 7.00M/8.34M [00:00<00:00, 69.0MB/s]
100% 8.34M/8.34M [00:00<00:00, 53.6MB/s]


In [9]:
!mv 'Microcontroller Detection' microcontroller-detection

In [10]:
%cd 'microcontroller-detection'
!wget https://raw.githubusercontent.com/TannerGilbert/YOLO-Tutorials/master/YOLO-Object-Detection-with-Darknet/convert_voc_to_yolo.py

/content/darknet/microcontroller-detection
--2021-03-13 07:45:49--  https://raw.githubusercontent.com/TannerGilbert/YOLO-Tutorials/master/YOLO-Object-Detection-with-Darknet/convert_voc_to_yolo.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2064 (2.0K) [text/plain]
Saving to: ‘convert_voc_to_yolo.py’

convert_voc_to_yolo 100%[===================>]   2.02K  --.-KB/s    in 0s      

2021-03-13 07:45:49 (43.5 MB/s) - ‘convert_voc_to_yolo.py’ saved [2064/2064]



In [11]:
!python convert_voc_to_yolo.py

Finished processing: train
Finished processing: test


In [12]:
classes_file = open('custom.names', 'w')
classes = ['Arduino_Nano', 'Heltec_ESP32_Lora', 'ESP8266', 'Raspberry_Pi_3']
num_classes = len(classes)
for c in classes:
  classes_file.write(c + '\n')
classes_file.close()

In [13]:
%cd /content/darknet

/content/darknet


In [14]:
config_file = open('detector.data', 'w')

config_file.write(f'classes={num_classes}\n')
config_file.write('train=/content/darknet/microcontroller-detection/train.txt\n')
config_file.write('valid=/content/darknet/microcontroller-detection/test.txt\n')
config_file.write('names=/content/darknet/microcontroller-detection/custom.names\n')
config_file.write('backup=backup/')

config_file.close()

In [15]:
!cat detector.data

classes=4
train=/content/darknet/microcontroller-detection/train.txt
valid=/content/darknet/microcontroller-detection/test.txt
names=/content/darknet/microcontroller-detection/custom.names
backup=backup/

In [16]:
!cp cfg/yolov3.cfg yolov3-custom.cfg
with open('yolov3-custom.cfg', 'r') as file_:
  file_content = file_.read()
file_content = file_content.replace('batch=1', 'batch=64').replace('subdivisions=1', 'subdivisions=16').replace('max_batches = 500200', 'max_batches = 4000').replace('steps=400000,450000', 'steps=3200,3600').replace('classes=80', f'classes={num_classes}').replace('filters=255', f'filters={(num_classes + 5) * 3}')

f = open('yolov3-custom.cfg', 'w')
f.write(file_content)
f.close()

In [17]:
!cat yolov3-custom.cfg

[net]
# Testing
batch=64
subdivisions=16
# Training
# batch=64
# subdivisions=166
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 4000
policy=steps
steps=3200,3600
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

# Downsample

[convolutional]
batch_normalize=1
filters=128
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

[convolutional]


In [18]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-03-13 07:45:50--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  42.7MB/s    in 4.0s    

2021-03-13 07:45:54 (38.7 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [19]:
!./darknet detector train /content/darknet/detector.data /content/darknet/yolov3-custom.cfg /content/darknet/darknet53.conv.74 -dont_show

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
 total_bbox = 8219, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.328202), count: 4, class_loss = 131.868225, iou_loss = 2.376282, total_loss = 134.244507 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 304.278839, iou_loss = 0.000000, total_loss = 304.278839 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 696.539978, iou_loss = 0.000000, total_loss = 696.539978 
 total_bbox = 8223, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.374145), count: 4, class_loss = 132.035370, iou_loss = 3.334885, total_loss = 135.370255 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 305.340698, iou_loss = 0.000000,